In [4]:
###########################################################
### PROCESS FROM LINKEDIN STAGING INTO CORE DB TABLES   ###
### Version - Final                                     ###
### WRITTEN BY DANA CAINES - PROD DATE = 12/12/23       ###
###########################################################

import mysql.connector
from mysql.connector import Error

def create_db_connection():
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            passwd='rusercaines',
            database='smc_alumni_test'
            # host='34.27.128.119',
            # user='ahmed',
            # passwd='abcdefgh',
            # database='smc_alumni'
        )
        return conn
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
        return None


def insert_error_log(cursor, conn, error_data_tuple, source_function):
    insert_stmt = """INSERT INTO error_log (
        source, error_message, alumni_id, ...
    ) VALUES (%s, %s, %s, ...)
    """
    error_tuple = (source_function,) + error_data_tuple
    cursor.execute(insert_stmt, error_tuple)
    conn.commit()
    
    
def process_contact_information():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            
            # Processing logic for contact_information
            select_query = """
                SELECT alumniID, email_first, email_second, phone, company_phone 
                FROM stg_in_linkedin
                #LIMIT 1
            """
            cursor.execute(select_query)
            records = cursor.fetchall()
            
            for record in records:
                alumni_id, email_primary, email_secondary, phone, work_phone = record

                # Check if alumniID exists in contact_information
                cursor.execute("""
                    SELECT EXISTS(
                        SELECT 1 FROM contact_information
                        WHERE alumniID = %s
                    )
                """, (alumni_id,))
                record_exists = cursor.fetchone()[0]
                
                if record_exists:
                    # Update existing record
                    update_query = """
                        UPDATE contact_information
                        SET email_primary = COALESCE(NULLIF(%s, ''), email_primary),
                            email_secondary = COALESCE(NULLIF(%s, ''), email_secondary),
                            phone = COALESCE(NULLIF(%s, ''), phone),
                            company_phone = COALESCE(NULLIF(%s, ''), work_phone)
                        WHERE alumniID = %s
                    """
                    cursor.execute(update_query, (email_primary, email_secondary, phone, work_phone, alumni_id))
                else:
                    # Insert new record
                    insert_query = """
                        INSERT INTO contact_information (
                            alumniID, email_primary, email_secondary, phone, company_phone
                        ) VALUES (%s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_query, (alumni_id, email_primary, email_secondary, phone, work_phone))

                conn.commit()
          
            cursor.close()
        except Error as e:
            print("Error while processing contact_information:", e)
            conn.rollback()
        finally:
            if conn.is_connected():
                conn.close()

def process_employment():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            
            # Select data from stage table
            select_query = """
                SELECT alumniID, current_title, current_company, city, state, country
                FROM stg_in_linkedin
            """
            cursor.execute(select_query)
            records = cursor.fetchall()
            
            for record in records:
                alumni_id, job_title, employer, city, state, country = record

                # Check if a record with this combination exists in employment
                cursor.execute("""
                    SELECT EXISTS(
                        SELECT 1 FROM employment
                        WHERE alumniID = %s AND job_title = %s AND employer = %s AND city = %s
                    )
                """, (alumni_id, job_title, employer, city))
                record_exists = cursor.fetchone()[0]
                
                if record_exists:
                    # Update existing record
                    update_query = """
                        UPDATE employment
                        SET job_title = COALESCE(NULLIF(%s, ''), job_title),
                            employer = COALESCE(NULLIF(%s, ''), employer),
                            city = COALESCE(NULLIF(%s, ''), city),
                            state = COALESCE(NULLIF(%s, ''), state),
                            country = COALESCE(NULLIF(%s, ''), country)
                        WHERE alumniID = %s AND job_title = %s AND employer = %s AND city = %s
                    """
                    cursor.execute(update_query, (job_title, employer, city, state, country, alumni_id, job_title, employer, city))
                else:
                    # Insert new record
                    insert_query = """
                        INSERT INTO employment (
                            alumniID, job_title, employer, city, state, country
                        ) VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_query, (alumni_id, job_title, employer, city, state, country))

                conn.commit()

        except Error as e:
            print("Error while processing employment:", e)
            error_data_tuple = (str(e),) + record
            insert_error_log(cursor, conn, error_data_tuple, "process_employment")
            conn.rollback()
        finally:
            if conn.is_connected():
                conn.close()

                
def process_education():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()

            # Select data from stage table
            select_query = """
                SELECT alumniID, sm_school, degree2, field_of_study_2, school_2, start_year_2, end_year_2, years_school_2,
                       degree3, field_of_study_3, school_3, start_year_3, end_year_3, years_school_3,
                       degree4, field_of_study_4, school_4, start_year_4, end_year_4, years_school_4
                FROM stg_in_linkedin
            """
            cursor.execute(select_query)
            records = cursor.fetchall()

            for record in records:
                alumni_id, sm_school, *education_data = record

                # Process each education set (2, 3, 4)
                for i in range(0, 18, 6):
                    degree, field_of_study, school, start_year, end_year, years_attended = education_data[i:i+6]

                    if degree and field_of_study and school:
                        # Check if a record with this combination exists in education
                        cursor.execute("""
                            SELECT EXISTS(
                                SELECT 1 FROM education
                                WHERE alumniID = %s AND degree = %s AND field_of_study = %s AND school = %s
                            )
                        """, (alumni_id, degree, field_of_study, school))
                        record_exists = cursor.fetchone()[0]

                        if record_exists:
                            # Update existing record
                            update_query = """
                                UPDATE education
                                SET start_year = COALESCE(NULLIF(%s, ''), start_year),
                                    end_year = COALESCE(NULLIF(%s, ''), end_year),
                                    years_attended = COALESCE(NULLIF(%s, ''), years_attended)
                                WHERE alumniID = %s AND degree = %s AND field_of_study = %s AND school = %s
                            """
                            cursor.execute(update_query, (start_year, end_year, years_attended, alumni_id, degree, field_of_study, school))
                        else:
                            # Insert new record
                            insert_query = """
                                INSERT INTO education (
                                    alumniID, sm_school, degree, field_of_study, school, start_year, end_year, years_attended
                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                            """
                            cursor.execute(insert_query, (alumni_id, sm_school if i == 0 else None, degree, field_of_study, school, start_year, end_year, years_attended))

                        conn.commit()

        except Error as e:
            print("Error while processing education:", e)
            if record:
                error_data_tuple = (str(e),) + record
            else:
                error_data_tuple = (str(e),)
            insert_error_log(cursor, conn, error_data_tuple, "process_education")
            conn.rollback()
        finally:
            if conn is not None and conn.is_connected():
                conn.close()

# Call the functions to process data
#process_contact_information()
#process_employment()
#process_education()
